In [40]:
import pandas as pd
import numpy as np
from sklearn import datasets

In [10]:
wine = pd.DataFrame(datasets.load_wine()['data'])
wine.columns = datasets.load_wine().feature_names

# Manipulação de dados em pandas para usuários de R
A ideia aqui é trazer um pouco da minha experiência com manipulação de dados nas duas linguagens de programação, facilitando a migração principalmente de usuários de dplyr que estão agora usando pandas para a manipulação de dados.

O formato das duas linguagens é naturalmente diferente, mas é possível adaptar a forma de aplicar as manipulações feitas em R para algo semelhante em pandas.

O objetivo desse documento é tentar trazer um pouco das equivalências que podem ser feitas no pandas para quem veio do dplyr.

Espero que ajude que passar por aqui!

## Criando colunas 
Para criar colunas, no R usamos normalmente a função `dplyr::mutate`. Embora seja possível realizar essa tarefa sem usar nenhuma biblioteca, no contexto de manipulação de dados, o uso de `mutate` é preferível. No pandas, apesar de um pouco mais verboso, podemos usar o `assign`, que tem funcionalidade semelhante.

Como exemplo, vou usar o dataset `wine`, da biblioteca `sklearn`. 

Suponha que eu queira criar colunas com a razão de alguns componentes, como magnésio e flavonóides, pela concentração de álcool. A forma comum seria usando o cálculo através de colunas diretamente, mas isso também pode ser feito com `assign`.


In [15]:
wine.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0


In [16]:
wine.describe()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
count,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000
mean,13.000618,2.336348,2.366517,19.494944,99.741573,2.295112,2.029270,0.361854,1.590899,5.058090,0.957449,2.611685,746.893258
std,0.811827,1.117146,0.274344,3.339564,14.282484,0.625851,0.998859,0.124453,0.572359,2.318286,0.228572,0.709990,314.907474
min,11.030000,0.740000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.410000,1.280000,0.480000,1.270000,278.000000
25%,12.362500,1.602500,2.210000,17.200000,88.000000,1.742500,1.205000,0.270000,1.250000,3.220000,0.782500,1.937500,500.500000
50%,13.050000,1.865000,2.360000,19.500000,98.000000,2.355000,2.135000,0.340000,1.555000,4.690000,0.965000,2.780000,673.500000
75%,13.677500,3.082500,2.557500,21.500000,107.000000,2.800000,2.875000,0.437500,1.950000,6.200000,1.120000,3.170000,985.000000
max,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,13.000000,1.710000,4.000000,1680.000000


**Forma comum de criar colunas em python**

In [ ]:
wine['magnesium_by_alcohol'] = wine['magnesium'] / wine['alcohol']
wine['flavanoids_by_alcohol'] = wine['flavanoids'] / wine['alcohol']

**Criando colunas com pandas**

In [ ]:
wine.assign(
    magnesium_by_alcohol = lambda dff: dff['magnesium'] / dff['alcohol'],
    flavanoids_by_alcohol = lambda dff: dff['flavanoids'] / dff['alcohol']
)

Perceba que para usar assign, você também tem que usar uma `lambda function`. A `lambda function` será uma grande aliada na manipulação de dados, principalmente quando você precisa fazer transformações mais complexas. Nesses dois casos, você pode ter a percepção visual que a forma comum de criar as colunas é mais rápida de se fazer por exigir menos caracteres. Realmente exige menos caracteres, mas a segunda forma exige menos caracteres especiais, que, pelo menos para mim, acaba fazendo com que a segunda seja mais rápido de escrever. 

Concordo que realmente há o inconveniente de se usar o recurso da função lambda, mas acho que para quem vem do dplyr essa pode ser uma forma de criar colunas mais próxima ao `mutate`. Na verdade, para usuários mais atentos do `dplyr`, esse método é mais equivalente ao `transform`, pois uma coluna criada que foi criada no `assign` não pode ser referenciada logo em seguida, como seria possível no `mutate`. Para isso você precisaria criar outro `assign`. Retomarei esse ponto mais adiante ao falar de como substituir o `pipe`, muito usado na manipulação de dados com R.

### Criando colunas de categorias
Para criar colunas de categorias baseado em valores de outra coluna, no `dplyr` há algumas formas de fazer isso, sendo que o `ifelse` provavelmente seja a mais comum para criação de uma coluna com poucas categorias, mas também vemos, por vezes, o uso de `case_when` quando precisamos criar um número maior de categorias ou queremos deixar o código mais legível.

Há diversas formas de fazer isso em python. Vou apresentar a seguir as formas que vejo mais frequentemente. Como exemplo, vou criar duas colunas:
- Uma coluna baseado na coluna `ash`, que dirá se o valor está acima ou abaixo da média dos valores da coluna inteira
- Uma coluna de categorias baseado nos quartis da coluna `nonflavanoid_phenols`, que chamarei de `nfp_cat`

A primeira delas é com `np.where`, que seria o equivalente ao `ifelse` do dplyr:

In [45]:
wine.assign(
    ash_comparison = lambda dff: np.where(dff['ash'] < 2.366517, 'abaixo_media', 'acima_media'),
    nfp_cat = lambda dff: np.where(dff['nonflavanoid_phenols'] < 0.27, 'primeiro_quartil',
                                  np.where(dff['nonflavanoid_phenols'] < 0.34, 'segundo_quartil',
                                           np.where(dff['nonflavanoid_phenols'] < 0.4375, 'terceiro_quartil', 'quarto_quartil')))
).head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,ash_comparison,nfp_cat
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,acima_media,segundo_quartil
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,abaixo_media,primeiro_quartil
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,acima_media,segundo_quartil
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,acima_media,primeiro_quartil
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,acima_media,terceiro_quartil


Agora, mostrando o mesmo processo com `np.select`, que seria equivalente ao `case_when` usado no R. Os parâmetros do `np.select` são basicamente uma lista de condições, uma lista de categorias para cada condição, na mesma ordem das condições e, opcionalmente, um valor padrão/default. Se você não colocar nenhum valor como default e o valor não atender nenhuma das condições, a função preencherá a linha com o valor 0 (zero).

In [57]:
pd.__version__

'1.4.2'

In [61]:
wine_cat = wine.assign(
    ash_comparison = lambda dff: np.select([dff['ash'] < 2.366517], ['abaixo_media'], 'acima_media'), # aqui cloquei como padrão estar acima da média
    nfp_cat = lambda dff: np.select(
            [
                dff['nonflavanoid_phenols'] < 0.27,
                dff['nonflavanoid_phenols'] < 0.34,
                dff['nonflavanoid_phenols'] < 0.4375,
                dff['nonflavanoid_phenols'] >= 0.4375,
            ],
            [
                'primeiro_quartil',
                'segundo_quartil',
                'terceiro_quartil',
                'quarto_quartil'
            ],
            np.nan # aqui coloquei o valor vazio - np.nan equivale a vazio no pandas e não somente como um "not a number"
    )
) 